In [11]:
import pandas as pd
import os
import re
import sys
import numpy as np
from merf import MERF
import matplotlib.pyplot as plt
import seaborn as sns
import itertools 
sns.set_context("poster")
from sklearn.ensemble import RandomForestRegressor
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (11,8)
from merf.merf import MERF
from sklearn.model_selection import train_test_split, KFold
from merf.viz import plot_merf_training_stats

current_dir = os.getcwd() # Get the current working directory
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
from em_utils import * # import the utils

In [12]:
# Create output directory if it doesn't exist
output_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/merf_plots"
df_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/merf_dfs/"
os.makedirs(output_dir, exist_ok=True)

print("---------- Read metadata ----------")
m1_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/zachs_rerun/drift_fs/csv/all_omic_processed_data/deltas/"
train = read_data(m1_dir, "jan30_all_delta_train_imp_varcheck.csv")
test = read_data(m1_dir, "jan30_all_delta_test_imp_varcheck.csv")

print(test.columns.to_list() == test.columns.to_list())
print(train.columns.to_list() == test.columns.to_list())

print("Train columns :", train.columns.to_list())
print("Test columns:", test.columns.to_list())

---------- Read metadata ----------
True
True
Train columns : ['subject_id', 'randomized_group', 'age', 'sex', 'race', 'BMI', 'Weight', 'CRP', 'cholesterol', 'ghrelin', 'HDL', 'LDL', 'HbA1C', 'insulin', 'leptin', 'peptide_yy', 'tgcyd', 'homo_ir', 'DENOVOPURINE2-PWY', 'DHGLUCONATE-PYR-CAT-PWY', 'DTDPRHAMSYN-PWY', 'ECASYN-PWY', 'ENTBACSYN-PWY', 'FAO-PWY', 'FASYN-ELONG-PWY', 'FASYN-INITIAL-PWY', 'FERMENTATION-PWY', 'FOLSYN-PWY', 'FUC-RHAMCAT-PWY', 'FUCCAT-PWY', 'GALACT-GLUCUROCAT-PWY', 'GALACTARDEG-PWY', 'GALACTUROCAT-PWY', 'GALLATE-DEGRADATION-I-PWY', 'GALLATE-DEGRADATION-II-PWY', 'GLCMANNANAUT-PWY', 'GLUCARDEG-PWY', 'GLUCARGALACTSUPER-PWY', 'GLUCONEO-PWY', 'GLUCOSE1PMETAB-PWY', 'GLUCUROCAT-PWY', 'GLUTORN-PWY', 'GLYCOCAT-PWY', 'GLYCOGENSYNTH-PWY', 'GLYCOL-GLYOXDEG-PWY', 'GLYCOLYSIS', 'GLYCOLYSIS-E-D', 'GLYCOLYSIS-TCA-GLYOX-BYPASS', 'GLYOXYLATE-BYPASS', 'GOLPDLCAT-PWY', 'HCAMHPDEG-PWY', 'HEME-BIOSYNTHESIS-II', 'HEMESYN2-PWY', 'HEXITOLDEGSUPER-PWY', 'HISDEG-PWY', 'HISTSYN-PWY', 'HOMOSER-ME

In [13]:
# columns_to_drop = ['Weight', 'randomized_group', 'age', 'sex', 'race', 'BMI', 'CRP', 'cholesterol', 'ghrelin', 'HDL', 'LDL', 'HbA1C', 'insulin', 'leptin', 'peptide_yy', 'tgcyd', 'homo_ir']
columns_to_drop = ['BMI', 'HbA1C', 'tgcyd', 'cholesterol', 'ghrelin', 'leptin', 'peptide_yy', 'CRP']
train = train.drop([col for col in columns_to_drop if col in train.columns], axis=1)
test = test.drop([col for col in columns_to_drop if col in test.columns], axis=1)

In [14]:
print("test shape = ", test.shape)
print("train shape = ", train.shape)

test = test.dropna()
train = train.dropna()

print("test shape = ", test.shape)
print("train shape = ", train.shape)

test shape =  (62, 910)
train shape =  (234, 910)
test shape =  (62, 910)
train shape =  (234, 910)


In [15]:
print("---------- Select predictors for Basic training set ----------")
train_set = train
X = train_set.drop(['Weight', 'subject_id', 'range'], axis=1)
#X = X.drop(columns=['Unnamed: 0_tax', 'x_t'], errors='ignore')
Y = train_set[['Weight']]
Y = Y['Weight'].to_numpy() # Convert Y to numeric array
clusters_train = train_set['subject_id'].to_numpy() # Get ID variables
Z = np.ones((train_set.shape[0], 1)) # Create random effects matrix with ones
time = train_set['range'].astype(float).to_numpy() # Get time values as numeric array 

# Check the final columns
print("Final columns after drop:", X.columns.to_list())
print("X train values:", train_set['Weight'])

print("---------- Select predictors for Basic Raw test set ----------")
test_set = test
X_new = test_set.drop(['Weight', 'subject_id', 'range'], axis=1)
X_new = X_new[X.columns]  # Reorder and select columns to match training set
X_new = X_new.astype(X.dtypes)  # Ensure data types match

Y_new = test_set['Weight'].to_numpy()  # Convert Y to numeric array
clusters_new = pd.Series(test_set['subject_id'])  # Convert to pandas Series
# Create random effects matrix with ones
Z_new = np.ones((len(X_new), 1))
time_new = test_set['range'].astype(float).to_numpy()  # Convert time values to numeric array

---------- Select predictors for Basic training set ----------
Final columns after drop: ['randomized_group', 'age', 'sex', 'race', 'insulin', 'homo_ir', 'DENOVOPURINE2-PWY', 'DHGLUCONATE-PYR-CAT-PWY', 'DTDPRHAMSYN-PWY', 'ECASYN-PWY', 'ENTBACSYN-PWY', 'FAO-PWY', 'FASYN-ELONG-PWY', 'FASYN-INITIAL-PWY', 'FERMENTATION-PWY', 'FOLSYN-PWY', 'FUC-RHAMCAT-PWY', 'FUCCAT-PWY', 'GALACT-GLUCUROCAT-PWY', 'GALACTARDEG-PWY', 'GALACTUROCAT-PWY', 'GALLATE-DEGRADATION-I-PWY', 'GALLATE-DEGRADATION-II-PWY', 'GLCMANNANAUT-PWY', 'GLUCARDEG-PWY', 'GLUCARGALACTSUPER-PWY', 'GLUCONEO-PWY', 'GLUCOSE1PMETAB-PWY', 'GLUCUROCAT-PWY', 'GLUTORN-PWY', 'GLYCOCAT-PWY', 'GLYCOGENSYNTH-PWY', 'GLYCOL-GLYOXDEG-PWY', 'GLYCOLYSIS', 'GLYCOLYSIS-E-D', 'GLYCOLYSIS-TCA-GLYOX-BYPASS', 'GLYOXYLATE-BYPASS', 'GOLPDLCAT-PWY', 'HCAMHPDEG-PWY', 'HEME-BIOSYNTHESIS-II', 'HEMESYN2-PWY', 'HEXITOLDEGSUPER-PWY', 'HISDEG-PWY', 'HISTSYN-PWY', 'HOMOSER-METSYN-PWY', 'HSERMETANA-PWY', 'ILEUSYN-PWY', 'KDO-NAGLIPASYN-PWY', 'KETOGLUCONMET-PWY', 'LACTO

In [16]:
print("---------- MERF with finetuning RE 🌱 ----------")
import itertools 
import pandas as pd

print("---------- Select predictors for Basic Raw training set ----------")
train_set = train
X = train_set.drop(['Weight', 'subject_id', 'range'], axis=1)
print(f"X dimensions: {X.shape}")
y = train_set[['Weight']]
y = y['Weight'].to_numpy() # Convert Y to numeric array
print(f"y dimensions: {y.shape}")
clusters = train_set['subject_id'].to_numpy() # Get ID variables
print(f"clusters dimensions: {clusters.shape}")
z = np.ones((train_set.shape[0], 1)) # Create random effects matrix with ones
print(f"z dimensions: {z.shape}")

# Hyperparameters to tune
param_grid = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [None],
    'min_samples_split': [0.05, 0.1, 0.15],
    'max_iter': [2, 10],
    'n_splits': [3, 5, 10]  # Added n_splits for cross-validation
}

best_score = float('inf')
best_params = {}

# Initialize a list to store the results of each iteration
results = []

# Loop through all possible combinations of parameters
for params in itertools.product(*param_grid.values()):
        n_estimators, max_depth, min_samples_split, max_iter, n_splits = params
        print(f"Combination: {params}\n")
        scores = []
        prev = []
        ptev = []
        oob_scores = []  # Initialize a list to store OOB scores

        # K-fold cross-validation with variable n_splits
        kf = KFold(n_splits=n_splits)
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]  # Use .iloc for row selection
            y_train, y_test = y[train_index], y[test_index]
            clusters_train, clusters_test = clusters[train_index], pd.Series(clusters[test_index])
            z_train, z_test = z[train_index], z[test_index]
            model = MERF(
                # Specify the fixed effects model as a Random Forest Regressor
                fixed_effects_model=RandomForestRegressor(
                    n_estimators=n_estimators,  # Number of trees in the forest
                    max_depth=max_depth,  # Maximum depth of each tree
                    min_samples_split=min_samples_split,  # Minimum samples required to split an internal node
                    n_jobs=1,  # Number of jobs to run in parallel
                    oob_score=True  # Whether to use out-of-bag samples to estimate the R^2 on unseen data
                ),
                # Generalized Linear Model (GLM) early stopping threshold
                gll_early_stop_threshold=None,  # No early stopping threshold set
                # Maximum number of iterations for the MERF algorithm
                max_iterations=max_iter  # Maximum number of iterations to run the MERF algorithm
            )
            model.fit(X_train.select_dtypes(include=[np.number]), z_train, pd.Series(clusters_train), y_train)
            y_pred = model.predict(X_test, z_test, clusters_test)
            scores.append(np.mean((y_pred - y_test) ** 2)) # MSE
            
            # Calculate ptev and prev
            total_variance = np.var(y_test) #calculates the total variance of the predicted values
            random_effect_variance = np.var(y_test - y_pred)  # Variance of residuals
            fixed_effect_variance = total_variance - random_effect_variance

            ptev.append(np.mean(fixed_effect_variance / total_variance if total_variance > 0 else 0))
            prev.append(np.mean(random_effect_variance / total_variance if total_variance > 0 else 0))

            # Calculate OOB score
            forest = model.trained_fe_model
            oob_score = round(forest.oob_score_*100, 1)  # percent variation
            oob_scores.append(oob_score)  # Append OOB score to the list

            # Print ptev, prev, and OOB score for the current iteration
            print(f"Combination, ptev: {np.mean(ptev):.4f}, prev: {np.mean(prev):.4f}, OOB Score: {oob_score:.4f}")

        # Calculate the mean of the scores for the current combination of parameters
        mean_score = np.mean(scores)
        mean_prev = np.mean(prev)
        mean_ptev = np.mean(ptev)
        mean_oob_score = np.mean(oob_scores)  # Calculate the mean of OOB scores
        if mean_score < best_score:
            best_score = mean_score
            best_params = params

        # Append the results of the current iteration to the results list
        # Create a result dictionary with individual scores and mean scores
        result_dict = {
            'n_estimators': n_estimators,
            'max_depth': max_depth,
            'min_samples_split': min_samples_split,
            'max_iter': max_iter,
            'n_splits': n_splits,  # Added n_splits to the result dictionary
            'mse_score_1': scores[0] if len(scores) > 0 else None,
            'mse_score_2': scores[1] if len(scores) > 1 else None,
            'mse_score_3': scores[2] if len(scores) > 2 else None,
            'mse_score_4': scores[3] if len(scores) > 3 else None,
            'mse_score_5': scores[4] if len(scores) > 4 else None,
            'mse_score_6': scores[5] if len(scores) > 5 else None,
            'mse_score_7': scores[6] if len(scores) > 6 else None,
            'mse_score_8': scores[7] if len(scores) > 7 else None,
            'mse_score_9': scores[8] if len(scores) > 8 else None,
            'mse_score_10': scores[9] if len(scores) > 9 else None,
            'mean_mse_score': mean_score,
            'prev_1': prev[0] if len(prev) > 0 else None,
            'prev_2': prev[1] if len(prev) > 1 else None,
            'prev_3': prev[2] if len(prev) > 2 else None,
            'prev_4': prev[3] if len(prev) > 3 else None,
            'prev_5': prev[4] if len(prev) > 4 else None,
            'prev_6': prev[5] if len(prev) > 5 else None,
            'prev_7': prev[6] if len(prev) > 6 else None,
            'prev_8': prev[7] if len(prev) > 7 else None,
            'prev_9': prev[8] if len(prev) > 8 else None,
            'prev_10': prev[9] if len(prev) > 9 else None,
            'mean_prev': mean_prev,
            'ptev_1': ptev[0] if len(ptev) > 0 else None,
            'ptev_2': ptev[1] if len(ptev) > 1 else None,
            'ptev_3': ptev[2] if len(ptev) > 2 else None,
            'ptev_4': ptev[3] if len(ptev) > 3 else None,
            'ptev_5': ptev[4] if len(ptev) > 4 else None,
            'ptev_6': ptev[5] if len(ptev) > 5 else None,
            'ptev_7': ptev[6] if len(ptev) > 6 else None,
            'ptev_8': ptev[7] if len(ptev) > 7 else None,
            'ptev_9': ptev[8] if len(ptev) > 8 else None,
            'ptev_10': ptev[9] if len(ptev) > 9 else None,
            'mean_ptev': mean_ptev,
            'oob_1': oob_scores[0] if len(oob_scores) > 0 else None,
            'oob_2': oob_scores[1] if len(oob_scores) > 1 else None,
            'oob_3': oob_scores[2] if len(oob_scores) > 2 else None,
            'oob_4': oob_scores[3] if len(oob_scores) > 3 else None,
            'oob_5': oob_scores[4] if len(oob_scores) > 4 else None,
            'oob_6': oob_scores[5] if len(oob_scores) > 5 else None,
            'oob_7': oob_scores[6] if len(oob_scores) > 6 else None,
            'oob_8': oob_scores[7] if len(oob_scores) > 7 else None,
            'oob_9': oob_scores[8] if len(oob_scores) > 8 else None,
            'oob_10': oob_scores[9] if len(oob_scores) > 9 else None,
            'oob_score': mean_oob_score
        }
        # Append the result dictionary to the results list
        results.append(result_dict)

print("Best parameters:", best_params)
print("Best score:", best_score)

# Convert the results list to a DataFrame and save it to a CSV file
df_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/merf_dfs/6.two_timepoints_deltas"
results_df = pd.DataFrame(results)
results_df.to_csv(f'{df_dir}/Feb2_2times_omic_deltas_weight_remove_time_var_imputed_swapped.csv', index=False)

---------- MERF with finetuning RE 🌱 ----------
---------- Select predictors for Basic Raw training set ----------
X dimensions: (234, 907)
y dimensions: (234,)
clusters dimensions: (234,)
z dimensions: (234, 1)
Combination: (100, None, 0.05, 2, 3)



INFO     [merf.py:307] Training GLL is 368.90084610940926 at iteration 1.
INFO     [merf.py:307] Training GLL is 418.4309470958252 at iteration 2.


Combination, ptev: 0.0812, prev: 0.9188, OOB Score: 52.0000


INFO     [merf.py:307] Training GLL is 499.00929368978603 at iteration 1.
INFO     [merf.py:307] Training GLL is 550.6130164374431 at iteration 2.


Combination, ptev: 0.1497, prev: 0.8503, OOB Score: 33.4000


...

Combination, ptev: -0.1912, prev: 1.1912, OOB Score: 37.2000


INFO     [merf.py:307] Training GLL is 833.9309260843281 at iteration 1.
INFO     [merf.py:307] Training GLL is 869.8251822726584 at iteration 2.
INFO     [merf.py:307] Training GLL is 865.6459891233609 at iteration 3.
INFO     [merf.py:307] Training GLL is 859.6382872342782 at iteration 4.
INFO     [merf.py:307] Training GLL is 855.925620898677 at iteration 5.
INFO     [merf.py:307] Training GLL is 854.3288061880853 at iteration 6.
INFO     [merf.py:307] Training GLL is 853.1740290962291 at iteration 7.
INFO     [merf.py:307] Training GLL is 849.4865690001376 at iteration 8.
INFO     [merf.py:307] Training GLL is 845.1124644203342 at iteration 9.
INFO     [merf.py:307] Training GLL is 841.7978835439864 at iteration 10.


Combination, ptev: -0.4172, prev: 1.4172, OOB Score: 38.4000


INFO     [merf.py:307] Training GLL is 877.2464100189552 at iteration 1.
INFO     [merf.py:307] Training GLL is 896.1545223025889 at iteration 2.
INFO     [merf.py:307] Training GLL is 883.4538330380944 at iteration 3.
INFO     [merf.py:307] Training GLL is 872.1948749100474 at iteration 4.
INFO     [merf.py:307] Training GLL is 864.2411191483163 at iteration 5.
INFO     [merf.py:307] Training GLL is 859.1011026823436 at iteration 6.
INFO     [merf.py:307] Training GLL is 858.5735247657466 at iteration 7.
INFO     [merf.py:307] Training GLL is 856.5680582802868 at iteration 8.
INFO     [merf.py:307] Training GLL is 851.1076068926511 at iteration 9.
INFO     [merf.py:307] Training GLL is 847.8135893523665 at iteration 10.


Combination, ptev: -0.3477, prev: 1.3477, OOB Score: 31.7000
Best parameters: (500, None, 0.15, 10, 10)
Best score: 36.02374002391879
